In [ ]:
USE DATABASE CSV_FINAL_ASSIGNMENT;
CREATE SCHEMA IF NOT EXISTS GOLD;
USE SCHEMA GOLD;

CREATE OR REPLACE TABLE CSV_FINAL_ASSIGNMENT.GOLD.FACT_READMISSION AS
WITH v_encounter_base AS (
  /* Normalize encounter class on-the-fly from SILVER */
  SELECT
    e.encounter_id,
    e.patient_id,
    e.admit_ts,
    e.discharge_ts,
    e.preadmit_number,
    e.facility_id,
    e.source_system,
    DATEDIFF('minute', e.admit_ts, e.discharge_ts) / 1440.0 AS los_days,
    CASE
      WHEN UPPER(TRIM(e.encounter_class)) IN ('I','IP','INPATIENT') THEN 'INPATIENT'
      WHEN UPPER(TRIM(e.encounter_class)) IN ('O','OP','OUTPATIENT') THEN 'OUTPATIENT'
      WHEN UPPER(TRIM(e.encounter_class)) IN ('E','ED','EMERGENCY')  THEN 'EMERGENCY'
      WHEN UPPER(TRIM(e.encounter_class)) IN ('OBS','OBSERVATION')  THEN 'OBSERVATION'
      ELSE 'UNKNOWN'
    END AS encounter_class_norm
  FROM CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS e
),

idx_all AS (
  /* Index discharges from ANY class (exclude UNKNOWN), require a discharge timestamp */
  SELECT
    patient_id,
    encounter_id,
    discharge_ts,
    facility_id,
    encounter_class_norm AS origin_class
  FROM v_encounter_base
  WHERE discharge_ts IS NOT NULL
    AND encounter_class_norm <> 'UNKNOWN'
),

fup_by_class AS (
  /* First subsequent admit within 30 days per destination class + overall */
  SELECT
    i.encounter_id AS index_encounter_id,

    /* Destination-specific first admit timestamps */
    MIN(CASE WHEN n.encounter_class_norm = 'INPATIENT'   THEN n.admit_ts END) AS next_admit_ts_ip,
    MIN(CASE WHEN n.encounter_class_norm = 'OUTPATIENT'  THEN n.admit_ts END) AS next_admit_ts_op,
    MIN(CASE WHEN n.encounter_class_norm = 'EMERGENCY'   THEN n.admit_ts END) AS next_admit_ts_ed,
    MIN(CASE WHEN n.encounter_class_norm = 'OBSERVATION' THEN n.admit_ts END) AS next_admit_ts_obs,

    /* Any destination first admit within the window */
    MIN(n.admit_ts) AS next_admit_ts_any
  FROM idx_all i
  JOIN v_encounter_base n
    ON n.patient_id = i.patient_id
   AND n.admit_ts   > i.discharge_ts
   AND n.admit_ts   <= DATEADD('day', 30, i.discharge_ts)  -- change 30 if needed
   AND n.encounter_class_norm <> 'UNKNOWN'
  GROUP BY i.encounter_id
),

base AS (
  /* Combine index encounters with first follow-up timestamps and destination flags */
  SELECT
    i.encounter_id,
    i.patient_id,
    i.discharge_ts AS discharge_date,
    i.origin_class,

    /* Destination timestamps for audit */
    f.next_admit_ts_any,
    f.next_admit_ts_ip,
    f.next_admit_ts_op,
    f.next_admit_ts_ed,
    f.next_admit_ts_obs,

    /* Destination flags */
    CASE WHEN f.next_admit_ts_ip  IS NOT NULL THEN 1 ELSE 0 END AS readmit_30d_to_inpatient,
    CASE WHEN f.next_admit_ts_op  IS NOT NULL THEN 1 ELSE 0 END AS readmit_30d_to_outpatient,
    CASE WHEN f.next_admit_ts_ed  IS NOT NULL THEN 1 ELSE 0 END AS readmit_30d_to_emergency,
    CASE WHEN f.next_admit_ts_obs IS NOT NULL THEN 1 ELSE 0 END AS readmit_30d_to_observation,

    /* Overall any-to-any flag */
    CASE WHEN f.next_admit_ts_any IS NOT NULL THEN 1 ELSE 0 END AS readmit_30d_any
  FROM idx_all i
  LEFT JOIN fup_by_class f
    ON f.index_encounter_id = i.encounter_id
)

SELECT
  /* Keys and context */
  encounter_id,
  patient_id,
  discharge_date,
  origin_class,

  /* Overall 30-day readmission flag */
  readmit_30d_any AS readmitted_in_30days_flag,

  /* Destination 30-day flags */
  readmit_30d_to_inpatient,
  readmit_30d_to_outpatient,
  readmit_30d_to_emergency,
  readmit_30d_to_observation,

  /* Origin → Destination matrix flags */
  CASE WHEN origin_class = 'INPATIENT'   AND readmit_30d_to_inpatient   = 1 THEN 1 ELSE 0 END AS inpatient_to_inpatient_readmission_flag,
  CASE WHEN origin_class = 'INPATIENT'   AND readmit_30d_to_outpatient  = 1 THEN 1 ELSE 0 END AS inpatient_to_outpatient_readmission_flag,
  CASE WHEN origin_class = 'INPATIENT'   AND readmit_30d_to_emergency   = 1 THEN 1 ELSE 0 END AS inpatient_to_emergency_readmission_flag,
  CASE WHEN origin_class = 'INPATIENT'   AND readmit_30d_to_observation = 1 THEN 1 ELSE 0 END AS inpatient_to_observation_readmission_flag,

  CASE WHEN origin_class = 'OUTPATIENT'  AND readmit_30d_to_inpatient   = 1 THEN 1 ELSE 0 END AS outpatient_to_inpatient_readmission_flag,
  CASE WHEN origin_class = 'OUTPATIENT'  AND readmit_30d_to_outpatient  = 1 THEN 1 ELSE 0 END AS outpatient_to_outpatient_readmission_flag,
  CASE WHEN origin_class = 'OUTPATIENT'  AND readmit_30d_to_emergency   = 1 THEN 1 ELSE 0 END AS outpatient_to_emergency_readmission_flag,
  CASE WHEN origin_class = 'OUTPATIENT'  AND readmit_30d_to_observation = 1 THEN 1 ELSE 0 END AS outpatient_to_observation_readmission_flag,

  CASE WHEN origin_class = 'EMERGENCY'   AND readmit_30d_to_inpatient   = 1 THEN 1 ELSE 0 END AS emergency_to_inpatient_readmission_flag,
  CASE WHEN origin_class = 'EMERGENCY'   AND readmit_30d_to_outpatient  = 1 THEN 1 ELSE 0 END AS emergency_to_outpatient_readmission_flag,
  CASE WHEN origin_class = 'EMERGENCY'   AND readmit_30d_to_emergency   = 1 THEN 1 ELSE 0 END AS emergency_to_emergency_readmission_flag,
  CASE WHEN origin_class = 'EMERGENCY'   AND readmit_30d_to_observation = 1 THEN 1 ELSE 0 END AS emergency_to_observation_readmission_flag,

  CASE WHEN origin_class = 'OBSERVATION' AND readmit_30d_to_inpatient   = 1 THEN 1 ELSE 0 END AS observation_to_inpatient_readmission_flag,
  CASE WHEN origin_class = 'OBSERVATION' AND readmit_30d_to_outpatient  = 1 THEN 1 ELSE 0 END AS observation_to_outpatient_readmission_flag,
  CASE WHEN origin_class = 'OBSERVATION' AND readmit_30d_to_emergency   = 1 THEN 1 ELSE 0 END AS observation_to_emergency_readmission_flag,
  CASE WHEN origin_class = 'OBSERVATION' AND readmit_30d_to_observation = 1 THEN 1 ELSE 0 END AS observation_to_observation_readmission_flag,

  /* Optional: expose destination timestamps for tracing in Power BI */
  next_admit_ts_any,
  next_admit_ts_ip   AS next_admit_ts_to_inpatient,
  next_admit_ts_op   AS next_admit_ts_to_outpatient,
  next_admit_ts_ed   AS next_admit_ts_to_emergency,
  next_admit_ts_obs  AS next_admit_ts_to_observation
FROM base;

/*##########################################################################################################################*/

  /*##########################################################################################################################*/


In [ ]:
CREATE OR REPLACE TABLE CSV_FINAL_ASSIGNMENT.GOLD.DIM_PATIENT_READMISSION (
  patient_id         VARCHAR PRIMARY KEY,
  patient_name       VARCHAR,
  gender             VARCHAR,
  city               VARCHAR,
  state              VARCHAR,
  date_of_birth      DATE,
  gender_norm        VARCHAR,
  dob_year           NUMBER(4),
  dob_month          NUMBER(2),
  dob_day            NUMBER(2),
  dob_date_key       NUMBER(8)
);

INSERT INTO CSV_FINAL_ASSIGNMENT.GOLD.DIM_PATIENT_READMISSION (
  patient_id, patient_name, gender, city, state, date_of_birth,
  gender_norm, dob_year, dob_month, dob_day, dob_date_key
)
SELECT
  sp.patient_id,
  CONCAT(sp.first_name, ' ', sp.last_name) AS patient_name,  -- ✨ YOUR LOGIC EXACTLY
  NULLIF(TRIM(sp.gender), '') AS gender,
  NULLIF(TRIM(sp.city), '')   AS city,
  NULLIF(TRIM(sp.state), '')  AS state,
  sp.birth_date::DATE         AS date_of_birth,

  CASE
    WHEN UPPER(TRIM(sp.gender)) IN ('M','MALE') THEN 'MALE'
    WHEN UPPER(TRIM(sp.gender)) IN ('F','FEMALE') THEN 'FEMALE'
    WHEN UPPER(TRIM(sp.gender)) IN ('O','OTHER','NON-BINARY','NB') THEN 'OTHER'
    WHEN sp.gender IS NULL OR TRIM(sp.gender) = '' OR UPPER(TRIM(sp.gender)) IN ('U','UNKNOWN','UNSPECIFIED') THEN 'UNKNOWN'
    ELSE UPPER(TRIM(sp.gender))
  END AS gender_norm,

  YEAR(sp.birth_date::DATE)  AS dob_year,
  MONTH(sp.birth_date::DATE) AS dob_month,
  DAY(sp.birth_date::DATE)   AS dob_day,

  CASE
    WHEN sp.birth_date IS NOT NULL
      THEN TO_NUMBER(TO_CHAR(sp.birth_date::DATE, 'YYYYMMDD'))
    ELSE NULL
  END AS dob_date_key
FROM CSV_FINAL_ASSIGNMENT.SILVER.SILVER_PATIENT sp
JOIN (
  SELECT DISTINCT patient_id
  FROM CSV_FINAL_ASSIGNMENT.GOLD.FACT_READMISSION
) fr
  ON fr.patient_id = sp.patient_id;